# FinBERT Sentiment Analysis

In [1]:
!pip install setuptools==67.8.0
!pip install transformers feedparser --use-deprecated=legacy-resolver yfinance

error: externally-managed-environment

× This environment is externally managed
╰─> To install Python packages system-wide, try brew install
    xyz, where xyz is the package you are trying to
    install.
    
    If you wish to install a Python library that isn't in Homebrew,
    use a virtual environment:
    
    python3 -m venv path/to/venv
    source path/to/venv/bin/activate
    python3 -m pip install xyz
    
    If you wish to install a Python application that isn't in Homebrew,
    it may be easiest to use 'pipx install xyz', which will manage a
    virtual environment for you. You can install pipx with
    
    brew install pipx
    
    You may restore the old behavior of pip by passing
    the '--break-system-packages' flag to pip, or by adding
    'break-system-packages = true' to your pip.conf file. The latter
    will permanently disable this error.
    
    If you disable this error, we STRONGLY recommend that you additionally
    pass the '--user' flag to pip, or set 

In [2]:
import feedparser
import yfinance as yf
import datetime
import matplotlib.pyplot as plt
from transformers import pipeline
from statistics import mean
import requests

2024-11-04 11:51:40.614077: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-classification", model="ProsusAI/finbert")

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [4]:
# Function to get the ticker symbol for a company name
def get_ticker(company_name):
    try:
        ticker = yf.Ticker(company_name).ticker
        return ticker if ticker else None
    except Exception as e:
        print(f"Error looking up ticker: {e}")
        return None

In [5]:
# User input for company name and keyword
company_name = input("Enter the company name: ")
keyword = input("Enter a keyword to filter articles (e.g., 'meta'): ")

ticker = get_ticker(company_name)

if not ticker:
    print("Ticker not found. Please try a different company name.")
else:
    print(f"Using ticker: {ticker}")

    # Parse RSS feed for financial news
    
    rss_feeds = [
    f'https://finance.yahoo.com/rss/headline?s={ticker}', 
    # f'https://www.nasdaq.com/feed/rssoutbound?symbol={ticker}'
    ]
    
    articles = []

    # Collect articles from all RSS feeds
    for rss_url in rss_feeds:
        feed = feedparser.parse(rss_url)
        for entry in feed.entries:
            articles.append(entry)
            
#     # Collect articles from all RSS feeds
#     for rss_url in rss_feeds:
#         try:
#             # Fetch content from the RSS feed
#             response = requests.get(rss_url, headers={'User-Agent': 'Mozilla/5.0'})
            
#             # Parse the RSS feed content
#             feed = feedparser.parse(response.content)
            
#             # Check if the feed has entries
#             if not feed.entries:
#                 print(f"No articles found in {rss_url}.")
#                 continue
            
#             # Append each entry to articles list
#             for entry in feed.entries:
#                 articles.append(entry)
        
#         except Exception as e:
#             print(f"An error occurred with {rss_url}: {e}")

    # rss_url = f'https://finance.yahoo.com/rss/headline?s={ticker}'
    # feed = feedparser.parse(rss_url)

    total_score = 0
    num_articles = 0
    sentiment_scores = []

    for i, entry in enumerate(articles): #feed.entries):
        if keyword.lower() not in entry.summary.lower():
            continue

        print(f'Title: {entry.title}')
        print(f'Link: {entry.link}')
        print(f'Published: {entry.published}')
        print(f'Summary: {entry.summary}')

        sentiment = pipe(entry.summary)[0]
        sentiment_scores.append(sentiment["score"] if sentiment["label"] == 'positive' else -sentiment["score"])

        print(f'Sentiment: {sentiment["label"]}, score: {sentiment["score"]}')
        print('-' * 40)

        total_score += sentiment["score"] if sentiment["label"] == 'positive' else -sentiment["score"]
        num_articles += 1

    if num_articles > 0:
        final_score = total_score / num_articles
        overall_sentiment = "Positive" if final_score >= 0.15 else "Negative" if final_score <= -0.15 else "Neutral"
        print(f'Overall sentiment: {overall_sentiment} ({final_score:.2f})')

        # Plot sentiment trend over time
        plt.figure(figsize=(10, 5))
        plt.plot(range(len(sentiment_scores)), sentiment_scores, marker='o')
        plt.title('Sentiment Trend Over Articles')
        plt.xlabel('Article Number')
        plt.ylabel('Sentiment Score')
        plt.axhline(y=0, color='gray', linestyle='--')
        plt.show()

        # Anomaly detection based on the mean
        avg_sentiment = mean(sentiment_scores)
        anomaly_threshold = 2 * abs(avg_sentiment)
        anomalies = [(i, score) for i, score in enumerate(sentiment_scores) if abs(score - avg_sentiment) > anomaly_threshold]

        if anomalies:
            print("\nAnomalies detected in sentiment scores:")
            for idx, score in anomalies:
                print(f"Article {idx + 1}: Sentiment Score = {score:.2f}")
        else:
            print("\nNo significant anomalies detected.")
    else:
        print("No articles matched the keyword.")


Using ticker: GS
No articles matched the keyword.
